# 跃迁误差抑制仿真 



## 1. 评估失谐微波驱动的影响

本部分旨在仿真一个失谐微波脉冲对量子比特的影响。我们将构建一个由失谐微波脉冲（MW）和一段空闲时间（idle）构成的序列单元，并将这个单元重复 `n` 次。

通过使用 `Scanner` 扫描 `idle` 时间的长度，我们可以观察到末态布居数随空闲时间的振荡，这个振荡频率应该与失谐量 `Delta` 相关。

**失谐弱驱动**

注意，驱动幅度不能太大，这里要仿真的就是驱动幅度较弱，必须要通过叠门才能看到的效应

### 1.1 研究脉冲时间的影响

#### 脉冲长度10ns

In [ ]:



# --- 0. 导入必要的库 ---
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, ptrace, Qobj
from qutip import Result

# 假设所有必要的类都已从您的库中导入
from quantum_sim import (DuffingOscillatorModel, Simulator, Scanner, 
                         CosineEnvelope, MicrowaveSequence, CompositeEnvelope)

# --- 1. 定义物理系统和脉冲参数 ---
# 物理系统参数
D_LEVELS = 2
OMEGA_Q_GHZ = 5.3
ANH_GHZ = -0.212

# 失谐微波脉冲 (MW) 参数
PULSE_DURATION = 10  # ns, 单个微波脉冲的时长
# PULSE_AMP_RAD_NS = (np.pi / PULSE_DURATION) # 幅度大致相当于 1 个 pi/2 脉冲
PULSE_AMP_RAD_NS = (np.pi / PULSE_DURATION) * 0.1
DETUNING_GHZ = 0.05 # 50 MHz 的失谐量

# --- 2. 仿真控制参数 ---
# 初始状态
PSI0 = basis(D_LEVELS, 0)

# 观测量：|1> 态的布居数
pop1_operator = basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag()

# 扫描和循环参数
n_values = [2, 3, 4, 10] # 序列单元的重复次数
# idle_scan_values = np.linspace(0, 1 / OMEGA_Q_GHZ * 5, 21) # 扫描的空闲时间范围 (ns)
idle_scan_values = np.linspace(0, 1 / DETUNING_GHZ, 51) # 扫描的空闲时间范围 (ns)



# 准备一个字典来存储每次扫描的结果
results_dict = {}

# --- 3. 创建共享的模型和仿真器实例 ---
model = DuffingOscillatorModel(
    d=D_LEVELS,
    omega_q=OMEGA_Q_GHZ * 2 * np.pi,
    anh=ANH_GHZ * 2 * np.pi
)

sim = Simulator(model=model, options={'nsteps': 50000, 'atol': 1e-9, 'rtol': 1e-7})

print("所有仿真组件已成功创建。准备开始多轮扫描...")

# --- 4. 主循环：遍历不同的脉冲重复次数 n ---

for n in n_values:
    print(f"\\n===== 开始扫描 n = {n} 的情况 =====")

    

    # b) 定义本次扫描中固定不变的参数
    base_args = {
        'psi0': PSI0,
        'use_rwa': False, # 失谐效应在实验室坐标系下更显著和准确
        'e_ops': [pop1_operator]
    }

    # c) 实例化 Scanner
    scanner = Scanner(simulator=sim, base_sim_args=base_args)

    # d) 定义本次扫描的 Setup 函数
    def detuning_setup(idle_duration: float) -> dict:
        # 1. 创建基础包络：失谐微波脉冲 (MW) 和 空闲时间 (idle)
        mw_envelope = CosineEnvelope(duration=PULSE_DURATION, amp=PULSE_AMP_RAD_NS)
        idle_envelope = CosineEnvelope(duration=idle_duration, amp=0.0)

        total_duration = n * (PULSE_DURATION + idle_duration)
        t_list = np.linspace(0, total_duration, int(total_duration * 50 + 1))

        # 2. 使用 '+' 和 '.repeat()' 构建完整的序列包络: (MW + idle) * n
        sequence_unit = mw_envelope + idle_envelope
        full_envelope = sequence_unit.repeat(n)
            
        # 3. 基于包络创建带有失谐的微波序列
        detuned_mw_sequence = MicrowaveSequence(
            envelope=full_envelope,
            carrier_freq=model.omega_q + (DETUNING_GHZ * 2 * np.pi)
        )

        return {'sequences': {'XY_drive': detuned_mw_sequence},
                't_list': t_list
            }

    # e) 定义 Post-process 函数
    def final_pop1_processor(idle_duration: float, result: Result) -> float:
        # 接收了 idle_duration 后，我们直接使用结果的最后一个点即可
        return result.expect[0][-1]

    # f) 执行对 idle 时间的扫描
    _, final_populations = scanner.run(
        sweep_values=idle_scan_values,
        setup_function=detuning_setup,
        post_process_function=final_pop1_processor,
        description=f"Scanning idle time for n={n}"
    )

    # g) 将结果存储到字典中
    results_dict[n] = final_populations

print("\\n...所有扫描完成。")

# --- 5. 结果统一可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))
# colors = plt.get_cmap('coolwarm')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    populations = results_dict[n]
    # color = colors(i / len(n_values))
    
    ax.plot(idle_scan_values, populations, 'o-', ms=4, label=f'n = {n} repetitions')

# 设置图表格式
ax.set_xlabel("Idle Time (ns)")
ax.set_ylabel("Final Population of |1> State")
ax.set_title(f"Detuned Rabi Oscillation (Detuning = {DETUNING_GHZ*1000} MHz)")
ax.grid(True)
ax.legend()
plt.show()

#### 脉冲长度13ns

In [ ]:



# --- 0. 导入必要的库 ---
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, ptrace, Qobj
from qutip import Result

# 假设所有必要的类都已从您的库中导入
from quantum_sim import (DuffingOscillatorModel, Simulator, Scanner, 
                         CosineEnvelope, MicrowaveSequence, CompositeEnvelope)

# --- 1. 定义物理系统和脉冲参数 ---
# 物理系统参数
D_LEVELS = 2
OMEGA_Q_GHZ = 5.3
ANH_GHZ = -0.212

# 失谐微波脉冲 (MW) 参数
PULSE_DURATION = 13  # ns, 单个微波脉冲的时长
# PULSE_AMP_RAD_NS = (np.pi / PULSE_DURATION) # 幅度大致相当于 1 个 pi/2 脉冲
PULSE_AMP_RAD_NS = (np.pi / PULSE_DURATION) * 0.1
DETUNING_GHZ = 0.05 # 50 MHz 的失谐量

# --- 2. 仿真控制参数 ---
# 初始状态
PSI0 = basis(D_LEVELS, 0)

# 观测量：|1> 态的布居数
pop1_operator = basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag()

# 扫描和循环参数
n_values = [2, 3, 4, 10] # 序列单元的重复次数
# idle_scan_values = np.linspace(0, 1 / OMEGA_Q_GHZ * 5, 21) # 扫描的空闲时间范围 (ns)
idle_scan_values = np.linspace(0, 1 / DETUNING_GHZ, 51) # 扫描的空闲时间范围 (ns)



# 准备一个字典来存储每次扫描的结果
results_dict = {}

# --- 3. 创建共享的模型和仿真器实例 ---
model = DuffingOscillatorModel(
    d=D_LEVELS,
    omega_q=OMEGA_Q_GHZ * 2 * np.pi,
    anh=ANH_GHZ * 2 * np.pi
)

sim = Simulator(model=model, options={'nsteps': 50000, 'atol': 1e-9, 'rtol': 1e-7})

print("所有仿真组件已成功创建。准备开始多轮扫描...")

# --- 4. 主循环：遍历不同的脉冲重复次数 n ---

for n in n_values:
    print(f"\\n===== 开始扫描 n = {n} 的情况 =====")

    

    # b) 定义本次扫描中固定不变的参数
    base_args = {
        'psi0': PSI0,
        'use_rwa': False, # 失谐效应在实验室坐标系下更显著和准确
        'e_ops': [pop1_operator]
    }

    # c) 实例化 Scanner
    scanner = Scanner(simulator=sim, base_sim_args=base_args)

    # d) 定义本次扫描的 Setup 函数
    def detuning_setup(idle_duration: float) -> dict:
        # 1. 创建基础包络：失谐微波脉冲 (MW) 和 空闲时间 (idle)
        mw_envelope = CosineEnvelope(duration=PULSE_DURATION, amp=PULSE_AMP_RAD_NS)
        idle_envelope = CosineEnvelope(duration=idle_duration, amp=0.0)

        total_duration = n * (PULSE_DURATION + idle_duration)
        t_list = np.linspace(0, total_duration, int(total_duration * 50 + 1))

        # 2. 使用 '+' 和 '.repeat()' 构建完整的序列包络: (MW + idle) * n
        sequence_unit = mw_envelope + idle_envelope
        full_envelope = sequence_unit.repeat(n)
            
        # 3. 基于包络创建带有失谐的微波序列
        detuned_mw_sequence = MicrowaveSequence(
            envelope=full_envelope,
            carrier_freq=model.omega_q + (DETUNING_GHZ * 2 * np.pi)
        )

        return {'sequences': {'XY_drive': detuned_mw_sequence},
                't_list': t_list
            }

    # e) 定义 Post-process 函数
    def final_pop1_processor(idle_duration: float, result: Result) -> float:
        # 接收了 idle_duration 后，我们直接使用结果的最后一个点即可
        return result.expect[0][-1]

    # f) 执行对 idle 时间的扫描
    _, final_populations = scanner.run(
        sweep_values=idle_scan_values,
        setup_function=detuning_setup,
        post_process_function=final_pop1_processor,
        description=f"Scanning idle time for n={n}"
    )

    # g) 将结果存储到字典中
    results_dict[n] = final_populations

print("\\n...所有扫描完成。")

# --- 5. 结果统一可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))
# colors = plt.get_cmap('coolwarm')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    populations = results_dict[n]
    # color = colors(i / len(n_values))
    
    ax.plot(idle_scan_values, populations, 'o-', ms=4, label=f'n = {n} repetitions')

# 设置图表格式
ax.set_xlabel("Idle Time (ns)")
ax.set_ylabel("Final Population of |1> State")
ax.set_title(f"Detuned Rabi Oscillation (Detuning = {DETUNING_GHZ*1000} MHz)")
ax.grid(True)
ax.legend()
plt.show()

注意到，放大峰的位置发生了变化。说明这个峰的位置的确是和脉冲长度有关的（关联到旋转坐标系中的等效Z旋转）

### 1.2 研究D的影响

#### D=2 简单二能级

In [ ]:



# --- 0. 导入必要的库 ---
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, ptrace, Qobj, qeye
from qutip import Result

# 假设所有必要的类都已从您的库中导入
from quantum_sim import (DuffingOscillatorModel, Simulator, Scanner, 
                         CosineEnvelope, MicrowaveSequence, CompositeEnvelope)

# --- 1. 定义物理系统和脉冲参数 ---
# 物理系统参数
D_LEVELS = 2
OMEGA_Q_GHZ = 5.3
ANH_GHZ = -0.212

# 失谐微波脉冲 (MW) 参数
PULSE_DURATION = 10  # ns, 单个微波脉冲的时长
# PULSE_AMP_RAD_NS = (np.pi / PULSE_DURATION) # 幅度大致相当于 1 个 pi/2 脉冲
PULSE_AMP_RAD_NS = (np.pi / PULSE_DURATION) * 0.4
DETUNING_GHZ = -0.08 # -80 MHz 的失谐量

# --- 2. 仿真控制参数 ---
# 初始状态
PSI0 = basis(D_LEVELS, 0)

# 观测量：|1> 态的布居数
pop1_operator = basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag()
# 观测量：|0> 态的布居数
pop0_operator = basis(D_LEVELS, 0) * basis(D_LEVELS, 0).dag()
leakage_operator = qeye(D_LEVELS) - pop0_operator - pop1_operator

# 扫描和循环参数
n_values = [10] # 序列单元的重复次数
# idle_scan_values = np.linspace(0, 1 / OMEGA_Q_GHZ * 5, 21) # 扫描的空闲时间范围 (ns)
idle_scan_values = np.linspace(0, np.abs(1 / DETUNING_GHZ), 51) # 扫描的空闲时间范围 (ns)



# 准备一个字典来存储每次扫描的结果
results_dict = {}

# --- 3. 创建共享的模型和仿真器实例 ---
model = DuffingOscillatorModel(
    d=D_LEVELS,
    omega_q=OMEGA_Q_GHZ * 2 * np.pi,
    anh=ANH_GHZ * 2 * np.pi
)

sim = Simulator(model=model, options={'nsteps': 50000, 'atol': 1e-9, 'rtol': 1e-7})

print("所有仿真组件已成功创建。准备开始多轮扫描...")

# --- 4. 主循环：遍历不同的脉冲重复次数 n ---

for n in n_values:
    print(f"\\n===== 开始扫描 n = {n} 的情况 =====")

    

    # b) 定义本次扫描中固定不变的参数
    base_args = {
        'psi0': PSI0,
        'use_rwa': False, # 失谐效应在实验室坐标系下更显著和准确
        'e_ops': [pop0_operator, pop1_operator, leakage_operator]
    }

    # c) 实例化 Scanner
    scanner = Scanner(simulator=sim, base_sim_args=base_args)

    # d) 定义本次扫描的 Setup 函数
    def detuning_setup(idle_duration: float) -> dict:
        # 1. 创建基础包络：失谐微波脉冲 (MW) 和 空闲时间 (idle)
        mw_envelope = CosineEnvelope(duration=PULSE_DURATION, amp=PULSE_AMP_RAD_NS)
        idle_envelope = CosineEnvelope(duration=idle_duration, amp=0.0)

        total_duration = n * (PULSE_DURATION + idle_duration)
        t_list = np.linspace(0, total_duration, int(total_duration * 50 + 1))

        # 2. 使用 '+' 和 '.repeat()' 构建完整的序列包络: (MW + idle) * n
        sequence_unit = mw_envelope + idle_envelope
        full_envelope = sequence_unit.repeat(n)
            
        # 3. 基于包络创建带有失谐的微波序列
        detuned_mw_sequence = MicrowaveSequence(
            envelope=full_envelope,
            carrier_freq=model.omega_q + (DETUNING_GHZ * 2 * np.pi)
        )

        return {'sequences': {'XY_drive': detuned_mw_sequence},
                't_list': t_list
            }

    # e) 定义 Post-process 函数
    def final_pop1_processor(idle_duration: float, result: Result) -> float:
        # 接收了 idle_duration 后，我们直接使用结果的最后一个点即可
        return result.expect[0][-1], result.expect[1][-1], result.expect[2][-1]

    # f) 执行对 idle 时间的扫描
    _, final_populations = scanner.run(
        sweep_values=idle_scan_values,
        setup_function=detuning_setup,
        post_process_function=final_pop1_processor,
        description=f"Scanning idle time for n={n}"
    )

    # g) 将结果存储到字典中
    results_dict[n] = final_populations

print("\\n...所有扫描完成。")

# --- 5. 结果统一可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))
# colors = plt.get_cmap('coolwarm')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    populations = results_dict[n]
    # color = colors(i / len(n_values))
    pop0_data = [info[0] for info in populations]
    pop1_data = [info[1] for info in populations]
    leakage_data = [info[2] for info in populations]

    ax.plot(idle_scan_values, pop0_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |0>')
    ax.plot(idle_scan_values, pop1_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |1>')
    ax.plot(idle_scan_values, leakage_data, 'o-', ms=4, label=f'n = {n} repetitions: Leakage')

# 设置图表格式
ax.set_xlabel("Idle Time (ns)")
ax.set_ylabel("Final Population of |0> State")
ax.set_title(f"Detuned Rabi Oscillation (Detuning = {DETUNING_GHZ*1000} MHz)")
ax.grid(True)
ax.legend()
plt.show()

#### D=3 考虑二态

In [ ]:



# --- 0. 导入必要的库 ---
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, ptrace, Qobj, qeye
from qutip import Result

# 假设所有必要的类都已从您的库中导入
from quantum_sim import (DuffingOscillatorModel, Simulator, Scanner, 
                         CosineEnvelope, MicrowaveSequence, CompositeEnvelope)

# --- 1. 定义物理系统和脉冲参数 ---
# 物理系统参数
D_LEVELS = 3
OMEGA_Q_GHZ = 5.3
ANH_GHZ = -0.212

# 失谐微波脉冲 (MW) 参数
PULSE_DURATION = 10  # ns, 单个微波脉冲的时长
# PULSE_AMP_RAD_NS = (np.pi / PULSE_DURATION) # 幅度大致相当于 1 个 pi/2 脉冲
PULSE_AMP_RAD_NS = (np.pi / PULSE_DURATION) * 0.4
DETUNING_GHZ = -0.08 # -80 MHz 的失谐量

# --- 2. 仿真控制参数 ---
# 初始状态
PSI0 = basis(D_LEVELS, 0)

# 观测量：|1> 态的布居数
pop1_operator = basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag()
# 观测量：|0> 态的布居数
pop0_operator = basis(D_LEVELS, 0) * basis(D_LEVELS, 0).dag()
leakage_operator = qeye(D_LEVELS) - pop0_operator - pop1_operator

# 扫描和循环参数
n_values = [10] # 序列单元的重复次数
# idle_scan_values = np.linspace(0, 1 / OMEGA_Q_GHZ * 5, 21) # 扫描的空闲时间范围 (ns)
idle_scan_values = np.linspace(0, np.abs(1 / DETUNING_GHZ), 51) # 扫描的空闲时间范围 (ns)



# 准备一个字典来存储每次扫描的结果
results_dict = {}

# --- 3. 创建共享的模型和仿真器实例 ---
model = DuffingOscillatorModel(
    d=D_LEVELS,
    omega_q=OMEGA_Q_GHZ * 2 * np.pi,
    anh=ANH_GHZ * 2 * np.pi
)

sim = Simulator(model=model, options={'nsteps': 50000, 'atol': 1e-9, 'rtol': 1e-7})

print("所有仿真组件已成功创建。准备开始多轮扫描...")

# --- 4. 主循环：遍历不同的脉冲重复次数 n ---

for n in n_values:
    print(f"\\n===== 开始扫描 n = {n} 的情况 =====")

    

    # b) 定义本次扫描中固定不变的参数
    base_args = {
        'psi0': PSI0,
        'use_rwa': False, # 失谐效应在实验室坐标系下更显著和准确
        'e_ops': [pop0_operator, pop1_operator, leakage_operator]
    }

    # c) 实例化 Scanner
    scanner = Scanner(simulator=sim, base_sim_args=base_args)

    # d) 定义本次扫描的 Setup 函数
    def detuning_setup(idle_duration: float) -> dict:
        # 1. 创建基础包络：失谐微波脉冲 (MW) 和 空闲时间 (idle)
        mw_envelope = CosineEnvelope(duration=PULSE_DURATION, amp=PULSE_AMP_RAD_NS)
        idle_envelope = CosineEnvelope(duration=idle_duration, amp=0.0)

        total_duration = n * (PULSE_DURATION + idle_duration)
        t_list = np.linspace(0, total_duration, int(total_duration * 50 + 1))

        # 2. 使用 '+' 和 '.repeat()' 构建完整的序列包络: (MW + idle) * n
        sequence_unit = mw_envelope + idle_envelope
        full_envelope = sequence_unit.repeat(n)
            
        # 3. 基于包络创建带有失谐的微波序列
        detuned_mw_sequence = MicrowaveSequence(
            envelope=full_envelope,
            carrier_freq=model.omega_q + (DETUNING_GHZ * 2 * np.pi)
        )

        return {'sequences': {'XY_drive': detuned_mw_sequence},
                't_list': t_list
            }

    # e) 定义 Post-process 函数
    def final_pop1_processor(idle_duration: float, result: Result) -> float:
        # 接收了 idle_duration 后，我们直接使用结果的最后一个点即可
        return result.expect[0][-1], result.expect[1][-1], result.expect[2][-1]

    # f) 执行对 idle 时间的扫描
    _, final_populations = scanner.run(
        sweep_values=idle_scan_values,
        setup_function=detuning_setup,
        post_process_function=final_pop1_processor,
        description=f"Scanning idle time for n={n}"
    )

    # g) 将结果存储到字典中
    results_dict[n] = final_populations

print("\\n...所有扫描完成。")

# --- 5. 结果统一可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))
# colors = plt.get_cmap('coolwarm')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    populations = results_dict[n]
    # color = colors(i / len(n_values))
    pop0_data = [info[0] for info in populations]
    pop1_data = [info[1] for info in populations]
    leakage_data = [info[2] for info in populations]

    ax.plot(idle_scan_values, pop0_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |0>')
    ax.plot(idle_scan_values, pop1_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |1>')
    ax.plot(idle_scan_values, leakage_data, 'o-', ms=4, label=f'n = {n} repetitions: Leakage')

# 设置图表格式
ax.set_xlabel("Idle Time (ns)")
ax.set_ylabel("Final Population of |0> State")
ax.set_title(f"Detuned Rabi Oscillation (Detuning = {DETUNING_GHZ*1000} MHz)")
ax.grid(True)
ax.legend()
plt.show()

In [ ]:
excitation_peak_index = np.argmax(pop1_data)
excitation_peak_time = idle_scan_values[excitation_peak_index]

print(f"Excitation peak occurs at idle time: {excitation_peak_time:.2f} ns with population {pop1_data[excitation_peak_index]:.4f}")


leakage_peak_index = np.argmax(leakage_data)
leakage_peak_time = idle_scan_values[leakage_peak_index]

print(f"Leakage peak occurs at idle time: {leakage_peak_time:.2f} ns with population {leakage_data[leakage_peak_index]:.4f}")

## 2. 研究half-DRAG对失谐激发的抑制

### 2.1 固定idle为跃迁峰位置，扫描alpha系数

#### 考虑D=2

In [ ]:
# --- 0. 导入库 ---
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, ptrace, Qobj
from qutip import Result
from quantum_sim import DRAGCorrector
# from quantum_sim import ...

# --- 1. 定义物理系统和脉冲参数 ---
#    (为了单元格的独立性，我们在此处重新定义)
D_LEVELS = 2
OMEGA_Q_GHZ = 5.3
ANH_GHZ = -0.212
PULSE_DURATION = 10
PULSE_AMP_RAD_NS = 0.4 * (np.pi / PULSE_DURATION) 
DETUNING_GHZ = -0.08 # -80 MHz 失谐

# --- 2. 仿真控制参数 ---
# 初始状态
PSI0 = basis(D_LEVELS, 0)

# 观测量：|0>, |1> 态布居数和总泄漏
pop0_operator = basis(D_LEVELS, 0) * basis(D_LEVELS, 0).dag()
pop1_operator = basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag()
leakage_operator = Qobj(np.eye(D_LEVELS)) - pop0_operator - pop1_operator

# 固定的实验参数
IDLE_TIME_FIXED = 2.25  # ns, 从之前的仿真中找到的误差最大点

# [核心修改] 我们现在要循环遍历不同的 n 值
n_values = [2, 5, 10] 

# 扫描参数
alpha_scan_values = np.linspace(-1.0, 3.0, 41) # 扫描 alpha 系数

# [核心修改] 准备一个字典来存储每次外层循环的结果
results_dict = {}


# --- 3. 创建共享的模型和仿真器实例 ---
model = DuffingOscillatorModel(d=D_LEVELS, omega_q=OMEGA_Q_GHZ*2*np.pi, anh=ANH_GHZ*2*np.pi)
sim = Simulator(model=model, options={'nsteps': 50000, 'atol': 1e-9, 'rtol': 1e-7})

print("开始多轮 DRAG alpha 系数扫描...")

# --- 4. [核心修改] 增加外层循环来遍历 n_values ---
for n_repetitions in n_values:
    print(f"\\n===== 开始扫描 n = {n_repetitions} 的情况 =====")

    # a) 动态计算本次扫描所需的仿真时间轴
    total_duration = n_repetitions * (PULSE_DURATION + IDLE_TIME_FIXED)
    t_list = np.linspace(0, total_duration, int(total_duration * 50 + 1))

    # b) 定义本次扫描中固定不变的参数
    base_args = {
        'psi0': PSI0,
        't_list': t_list,
        'use_rwa': False,
        'e_ops': [pop0_operator, pop1_operator, leakage_operator]
    }

    # c) 实例化 Scanner
    scanner = Scanner(simulator=sim, base_sim_args=base_args)

    # d) 定义本次扫描的 Setup 函数 (它会捕获当前的 n_repetitions)
    def hd_alpha_setup(alpha: float) -> dict:
        mw_envelope = CosineEnvelope(duration=PULSE_DURATION, amp=PULSE_AMP_RAD_NS)
        idle_envelope = CosineEnvelope(duration=IDLE_TIME_FIXED, amp=0.0)
        sequence_unit = mw_envelope + idle_envelope
        full_envelope = sequence_unit.repeat(n_repetitions)
        
        detuned_mw_sequence = MicrowaveSequence(
            envelope=full_envelope,
            carrier_freq=model.omega_q + (DETUNING_GHZ * 2 * np.pi)
        )
        
        drag_corrector = DRAGCorrector(
            anh=(- DETUNING_GHZ * 2 * np.pi), 
            alpha=alpha
        )
        dragged_sequence = drag_corrector.apply(detuned_mw_sequence)
        
        return {'sequences': {'XY_drive': dragged_sequence}}

    # e) 定义 Post-process 函数
    def final_populations_processor(alpha: float, result: Result) -> tuple:
        p0_final = result.expect[0][-1]
        p1_final = result.expect[1][-1]
        leakage_final = result.expect[2][-1]
        return (p0_final, p1_final, leakage_final)

    # f) 执行对 alpha 的扫描
    _, final_populations_list = scanner.run(
        sweep_values=alpha_scan_values,
        setup_function=hd_alpha_setup,
        post_process_function=final_populations_processor,
        description=f"Scanning alpha for n={n_repetitions}"
    )

    # g) 将结果存储到字典中
    results_dict[n_repetitions] = final_populations_list

print("\\n...所有扫描完成。")

# --- 5. [核心修改] 结果统一可视化，只绘制 P0 ---
fig, ax = plt.subplots(figsize=(12, 8))
colors = plt.get_cmap('viridis')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    # 提取 P0 的数据
    populations_list = results_dict[n]
    p0_values, _, _ = np.array(populations_list).T
    
    color = colors(i / len(n_values))
    ax.plot(alpha_scan_values, p0_values, 'o-', ms=4, color=color, label=f'Pop $|0\\rangle$ for n={n}')
    
    # 寻找最优 alpha (使 P0 最大化的点)
    optimal_alpha_index = np.argmax(p0_values)
    optimal_alpha = alpha_scan_values[optimal_alpha_index]
    max_fidelity = p0_values[optimal_alpha_index]
    ax.plot(optimal_alpha, max_fidelity, '*', color=color, markersize=15, 
            label=f'Optimal for n={n} (alpha={optimal_alpha:.3f})')

# 设置图表格式
ax.set_xlabel("DRAG Alpha Coefficient")
ax.set_ylabel("Final Population of |0> State (Fidelity)")
ax.set_title(f"DRAG Suppression of Detuning Error for different repetitions")
ax.grid(True)
ax.legend()
plt.show()

#### 考虑D=3

##### 将idle时间设置在01跃迁峰的最大值

In [ ]:
# --- 0. 导入库 ---
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, ptrace, Qobj
from qutip import Result
from quantum_sim import DRAGCorrector
# from quantum_sim import ...

# --- 1. 定义物理系统和脉冲参数 ---
#    (为了单元格的独立性，我们在此处重新定义)
D_LEVELS = 3
OMEGA_Q_GHZ = 5.3
ANH_GHZ = -0.212
PULSE_DURATION = 10
PULSE_AMP_RAD_NS = 0.4 * (np.pi / PULSE_DURATION) 
DETUNING_GHZ = -0.08 # -80 MHz 失谐

# --- 2. 仿真控制参数 ---
# 初始状态
PSI0 = basis(D_LEVELS, 0)

# 观测量：|0>, |1> 态布居数和总泄漏
pop0_operator = basis(D_LEVELS, 0) * basis(D_LEVELS, 0).dag()
pop1_operator = basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag()
leakage_operator = Qobj(np.eye(D_LEVELS)) - pop0_operator - pop1_operator

# 固定的实验参数
IDLE_TIME_FIXED = 2.25  # ns, 从之前的仿真中找到的误差最大点

# [核心修改] 我们现在要循环遍历不同的 n 值
n_values = [2, 5, 10] 

# 扫描参数
alpha_scan_values = np.linspace(-1.0, 3.0, 41) # 扫描 alpha 系数

# [核心修改] 准备一个字典来存储每次外层循环的结果
results_dict = {}


# --- 3. 创建共享的模型和仿真器实例 ---
model = DuffingOscillatorModel(d=D_LEVELS, omega_q=OMEGA_Q_GHZ*2*np.pi, anh=ANH_GHZ*2*np.pi)
sim = Simulator(model=model, options={'nsteps': 50000, 'atol': 1e-9, 'rtol': 1e-7})

print("开始多轮 DRAG alpha 系数扫描...")

# --- 4. [核心修改] 增加外层循环来遍历 n_values ---
for n_repetitions in n_values:
    print(f"\\n===== 开始扫描 n = {n_repetitions} 的情况 =====")

    # a) 动态计算本次扫描所需的仿真时间轴
    total_duration = n_repetitions * (PULSE_DURATION + IDLE_TIME_FIXED)
    t_list = np.linspace(0, total_duration, int(total_duration * 50 + 1))

    # b) 定义本次扫描中固定不变的参数
    base_args = {
        'psi0': PSI0,
        't_list': t_list,
        'use_rwa': False,
        'e_ops': [pop0_operator, pop1_operator, leakage_operator]
    }

    # c) 实例化 Scanner
    scanner = Scanner(simulator=sim, base_sim_args=base_args)

    # d) 定义本次扫描的 Setup 函数 (它会捕获当前的 n_repetitions)
    def hd_alpha_setup(alpha: float) -> dict:
        mw_envelope = CosineEnvelope(duration=PULSE_DURATION, amp=PULSE_AMP_RAD_NS)
        idle_envelope = CosineEnvelope(duration=IDLE_TIME_FIXED, amp=0.0)
        sequence_unit = mw_envelope + idle_envelope
        full_envelope = sequence_unit.repeat(n_repetitions)
        
        detuned_mw_sequence = MicrowaveSequence(
            envelope=full_envelope,
            carrier_freq=model.omega_q + (DETUNING_GHZ * 2 * np.pi)
        )
        
        drag_corrector = DRAGCorrector(
            anh=(- DETUNING_GHZ * 2 * np.pi), 
            alpha=alpha
        )
        dragged_sequence = drag_corrector.apply(detuned_mw_sequence)
        
        return {'sequences': {'XY_drive': dragged_sequence}}

    # e) 定义 Post-process 函数
    def final_populations_processor(alpha: float, result: Result) -> tuple:
        p0_final = result.expect[0][-1]
        p1_final = result.expect[1][-1]
        leakage_final = result.expect[2][-1]
        return (p0_final, p1_final, leakage_final)

    # f) 执行对 alpha 的扫描
    _, final_populations_list = scanner.run(
        sweep_values=alpha_scan_values,
        setup_function=hd_alpha_setup,
        post_process_function=final_populations_processor,
        description=f"Scanning alpha for n={n_repetitions}"
    )

    # g) 将结果存储到字典中
    results_dict[n_repetitions] = final_populations_list

print("\\n...所有扫描完成。")

# --- 5. [核心修改] 结果统一可视化，只绘制 P0 ---
fig, ax = plt.subplots(figsize=(12, 8))
colors = plt.get_cmap('viridis')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    # 提取 P0 的数据
    populations_list = results_dict[n]
    p0_values, _, _ = np.array(populations_list).T
    
    color = colors(i / len(n_values))
    ax.plot(alpha_scan_values, p0_values, 'o-', ms=4, color=color, label=f'Pop $|0\\rangle$ for n={n}')
    
    # 寻找最优 alpha (使 P0 最大化的点)
    optimal_alpha_index = np.argmax(p0_values)
    optimal_alpha = alpha_scan_values[optimal_alpha_index]
    max_fidelity = p0_values[optimal_alpha_index]
    ax.plot(optimal_alpha, max_fidelity, '*', color=color, markersize=15, 
            label=f'Optimal for n={n} (alpha={optimal_alpha:.3f})')

# 设置图表格式
ax.set_xlabel("DRAG Alpha Coefficient")
ax.set_ylabel("Final Population of |0> State (Fidelity)")
ax.set_title(f"DRAG Suppression of Detuning Error for different repetitions")
ax.grid(True)
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
colors = plt.get_cmap('viridis')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    # 提取 P0 的数据
    populations_list = results_dict[n]
    p0_values, p1_values, leakage_values = np.array(populations_list).T

    color = colors(i / len(n_values))
    ax.plot(alpha_scan_values, leakage_values, 'o-', ms=4, color=color, label=f'leakage for n={n}')

    # 寻找最优 alpha (使 P0 最大化的点)
    optimal_alpha_index = np.argmax(leakage_values)
    optimal_alpha = alpha_scan_values[optimal_alpha_index]
    max_fidelity = leakage_values[optimal_alpha_index]
    ax.plot(optimal_alpha, max_fidelity, '*', color=color, markersize=15, 
            label=f'Optimal for n={n} (alpha={optimal_alpha:.3f})')

# 设置图表格式
ax.set_xlabel("DRAG Alpha Coefficient")
ax.set_ylabel("Final Population of |0> State (Fidelity)")
ax.set_title(f"DRAG Suppression of Detuning Error for different repetitions")
ax.grid(True)
ax.legend()
plt.show()

In [ ]:
# --- 0. 导入库 ---
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, ptrace, Qobj
from qutip import Result
from quantum_sim import DRAGCorrector
# from quantum_sim import ...

# --- 1. 定义物理系统和脉冲参数 ---
#    (为了单元格的独立性，我们在此处重新定义)
D_LEVELS = 3
OMEGA_Q_GHZ = 5.3
ANH_GHZ = -0.212
PULSE_DURATION = 10
PULSE_AMP_RAD_NS = 0.4 * (np.pi / PULSE_DURATION) 
DETUNING_GHZ = -0.08 # -80 MHz 失谐

# --- 2. 仿真控制参数 ---
# 初始状态
PSI0 = basis(D_LEVELS, 0)

# 观测量：|0>, |1> 态布居数和总泄漏
pop0_operator = basis(D_LEVELS, 0) * basis(D_LEVELS, 0).dag()
pop1_operator = basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag()
leakage_operator = Qobj(np.eye(D_LEVELS)) - pop0_operator - pop1_operator

# 固定的实验参数
IDLE_TIME_FIXED = 2.25  # ns, 从之前的仿真中找到的误差最大点

# [核心修改] 我们现在要循环遍历不同的 n 值
n_values = [2, 5, 10] 

# 扫描参数
alpha_scan_values = np.linspace(-3.0, 3.0, 31) # 扫描 alpha 系数

# [核心修改] 准备一个字典来存储每次外层循环的结果
results_dict = {}


# --- 3. 创建共享的模型和仿真器实例 ---
model = DuffingOscillatorModel(d=D_LEVELS, omega_q=OMEGA_Q_GHZ*2*np.pi, anh=ANH_GHZ*2*np.pi)
sim = Simulator(model=model, options={'nsteps': 50000, 'atol': 1e-9, 'rtol': 1e-7})

print("开始多轮 DRAG alpha 系数扫描...")

# --- 4. [核心修改] 增加外层循环来遍历 n_values ---
for n_repetitions in n_values:
    print(f"\\n===== 开始扫描 n = {n_repetitions} 的情况 =====")

    # a) 动态计算本次扫描所需的仿真时间轴
    total_duration = n_repetitions * (PULSE_DURATION + IDLE_TIME_FIXED)
    t_list = np.linspace(0, total_duration, int(total_duration * 50 + 1))

    # b) 定义本次扫描中固定不变的参数
    base_args = {
        'psi0': PSI0,
        't_list': t_list,
        'use_rwa': False,
        'e_ops': [pop0_operator, pop1_operator, leakage_operator]
    }

    # c) 实例化 Scanner
    scanner = Scanner(simulator=sim, base_sim_args=base_args)

    # d) 定义本次扫描的 Setup 函数 (它会捕获当前的 n_repetitions)
    def hd_alpha_setup(alpha: float) -> dict:
        mw_envelope = CosineEnvelope(duration=PULSE_DURATION, amp=PULSE_AMP_RAD_NS)
        idle_envelope = CosineEnvelope(duration=IDLE_TIME_FIXED, amp=0.0)
        sequence_unit = mw_envelope + idle_envelope
        full_envelope = sequence_unit.repeat(n_repetitions)
        
        detuned_mw_sequence = MicrowaveSequence(
            envelope=full_envelope,
            carrier_freq=model.omega_q + (DETUNING_GHZ * 2 * np.pi)
        )
        
        drag_corrector = DRAGCorrector(
            anh=(- DETUNING_GHZ * 2 * np.pi), 
            alpha=alpha
        )
        dragged_sequence = drag_corrector.apply(detuned_mw_sequence)
        
        return {'sequences': {'XY_drive': dragged_sequence}}

    # e) 定义 Post-process 函数
    def final_populations_processor(alpha: float, result: Result) -> tuple:
        p0_final = result.expect[0][-1]
        p1_final = result.expect[1][-1]
        leakage_final = result.expect[2][-1]
        return (p0_final, p1_final, leakage_final)

    # f) 执行对 alpha 的扫描
    _, final_populations_list = scanner.run(
        sweep_values=alpha_scan_values,
        setup_function=hd_alpha_setup,
        post_process_function=final_populations_processor,
        description=f"Scanning alpha for n={n_repetitions}"
    )

    # g) 将结果存储到字典中
    results_dict[n_repetitions] = final_populations_list

print("\\n...所有扫描完成。")

# --- 5. 绘制 P0 ---
fig, ax = plt.subplots(figsize=(12, 8))
colors = plt.get_cmap('viridis')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    # 提取 P0 的数据
    populations_list = results_dict[n]
    p0_values, _, _ = np.array(populations_list).T
    
    color = colors(i / len(n_values))
    ax.plot(alpha_scan_values, p0_values, 'o-', ms=4, color=color, label=f'Pop $|0\\rangle$ for n={n}')
    
    # 寻找最优 alpha (使 P0 最大化的点)
    optimal_alpha_index = np.argmax(p0_values)
    optimal_alpha = alpha_scan_values[optimal_alpha_index]
    max_fidelity = p0_values[optimal_alpha_index]
    ax.plot(optimal_alpha, max_fidelity, '*', color=color, markersize=15, 
            label=f'Optimal for n={n} (alpha={optimal_alpha:.3f})')

# 设置图表格式
ax.set_xlabel("DRAG Alpha Coefficient")
ax.set_ylabel("Final Population of |0> State (Fidelity)")
ax.set_title(f"DRAG Suppression of Detuning Error for different repetitions")
ax.grid(True)
ax.legend()
plt.show()


# 绘制 Leakage
fig, ax = plt.subplots(figsize=(12, 8))
colors = plt.get_cmap('viridis')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    # 提取 P0 的数据
    populations_list = results_dict[n]
    p0_values, p1_values, leakage_values = np.array(populations_list).T

    color = colors(i / len(n_values))
    ax.plot(alpha_scan_values, leakage_values, 'o-', ms=4, color=color, label=f'leakage for n={n}')

    # 寻找最优 alpha (使 leakage 最小化的点)
    optimal_alpha_index = np.argmin(leakage_values)
    optimal_alpha = alpha_scan_values[optimal_alpha_index]
    min_leakage = leakage_values[optimal_alpha_index]
    ax.plot(optimal_alpha, min_leakage, '*', color=color, markersize=15, 
            label=f'Optimal for n={n} (alpha={optimal_alpha:.3f})')

# 设置图表格式
ax.set_xlabel("DRAG Alpha Coefficient")
ax.set_ylabel("Final Population of |0> State (Fidelity)")
ax.set_title(f"DRAG Suppression of Detuning Error for different repetitions")
ax.grid(True)
ax.legend()
plt.show()

##### 将idle时间设置在12跃迁峰的最大值

In [ ]:
# --- 0. 导入库 ---
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, ptrace, Qobj
from qutip import Result
from quantum_sim import DRAGCorrector
# from quantum_sim import ...

# --- 1. 定义物理系统和脉冲参数 ---
#    (为了单元格的独立性，我们在此处重新定义)
D_LEVELS = 3
OMEGA_Q_GHZ = 5.3
ANH_GHZ = -0.212
PULSE_DURATION = 10
PULSE_AMP_RAD_NS = 0.4 * (np.pi / PULSE_DURATION) 
DETUNING_GHZ = -0.08 # -80 MHz 失谐

# --- 2. 仿真控制参数 ---
# 初始状态
PSI0 = basis(D_LEVELS, 0)

# 观测量：|0>, |1> 态布居数和总泄漏
pop0_operator = basis(D_LEVELS, 0) * basis(D_LEVELS, 0).dag()
pop1_operator = basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag()
leakage_operator = Qobj(np.eye(D_LEVELS)) - pop0_operator - pop1_operator

# 固定的实验参数
IDLE_TIME_FIXED = 9.25  # ns, 从之前的仿真中找到的误差最大点

# [核心修改] 我们现在要循环遍历不同的 n 值
n_values = [2, 5, 10] 

# 扫描参数
alpha_scan_values = np.linspace(-8.0, 8.0, 41) # 扫描 alpha 系数

# [核心修改] 准备一个字典来存储每次外层循环的结果
results_dict = {}


# --- 3. 创建共享的模型和仿真器实例 ---
model = DuffingOscillatorModel(d=D_LEVELS, omega_q=OMEGA_Q_GHZ*2*np.pi, anh=ANH_GHZ*2*np.pi)
sim = Simulator(model=model, options={'nsteps': 50000, 'atol': 1e-9, 'rtol': 1e-7})

print("开始多轮 DRAG alpha 系数扫描...")

# --- 4. [核心修改] 增加外层循环来遍历 n_values ---
for n_repetitions in n_values:
    print(f"\\n===== 开始扫描 n = {n_repetitions} 的情况 =====")

    # a) 动态计算本次扫描所需的仿真时间轴
    total_duration = n_repetitions * (PULSE_DURATION + IDLE_TIME_FIXED)
    t_list = np.linspace(0, total_duration, int(total_duration * 50 + 1))

    # b) 定义本次扫描中固定不变的参数
    base_args = {
        'psi0': PSI0,
        't_list': t_list,
        'use_rwa': False,
        'e_ops': [pop0_operator, pop1_operator, leakage_operator]
    }

    # c) 实例化 Scanner
    scanner = Scanner(simulator=sim, base_sim_args=base_args)

    # d) 定义本次扫描的 Setup 函数 (它会捕获当前的 n_repetitions)
    def hd_alpha_setup(alpha: float) -> dict:
        mw_envelope = CosineEnvelope(duration=PULSE_DURATION, amp=PULSE_AMP_RAD_NS)
        idle_envelope = CosineEnvelope(duration=IDLE_TIME_FIXED, amp=0.0)
        sequence_unit = mw_envelope + idle_envelope
        full_envelope = sequence_unit.repeat(n_repetitions)
        
        detuned_mw_sequence = MicrowaveSequence(
            envelope=full_envelope,
            carrier_freq=model.omega_q + (DETUNING_GHZ * 2 * np.pi)
        )
        
        drag_corrector = DRAGCorrector(
            anh=(- DETUNING_GHZ * 2 * np.pi), 
            alpha=alpha
        )
        dragged_sequence = drag_corrector.apply(detuned_mw_sequence)
        
        return {'sequences': {'XY_drive': dragged_sequence}}

    # e) 定义 Post-process 函数
    def final_populations_processor(alpha: float, result: Result) -> tuple:
        p0_final = result.expect[0][-1]
        p1_final = result.expect[1][-1]
        leakage_final = result.expect[2][-1]
        return (p0_final, p1_final, leakage_final)

    # f) 执行对 alpha 的扫描
    _, final_populations_list = scanner.run(
        sweep_values=alpha_scan_values,
        setup_function=hd_alpha_setup,
        post_process_function=final_populations_processor,
        description=f"Scanning alpha for n={n_repetitions}"
    )

    # g) 将结果存储到字典中
    results_dict[n_repetitions] = final_populations_list

print("\\n...所有扫描完成。")

# --- 5. 绘制 P0 ---
fig, ax = plt.subplots(figsize=(12, 8))
colors = plt.get_cmap('viridis')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    # 提取 P0 的数据
    populations_list = results_dict[n]
    p0_values, _, _ = np.array(populations_list).T
    
    color = colors(i / len(n_values))
    ax.plot(alpha_scan_values, p0_values, 'o-', ms=4, color=color, label=f'Pop $|0\\rangle$ for n={n}')
    
    # 寻找最优 alpha (使 P0 最大化的点)
    optimal_alpha_index = np.argmax(p0_values)
    optimal_alpha = alpha_scan_values[optimal_alpha_index]
    max_fidelity = p0_values[optimal_alpha_index]
    ax.plot(optimal_alpha, max_fidelity, '*', color=color, markersize=15, 
            label=f'Optimal for n={n} (alpha={optimal_alpha:.3f})')

# 设置图表格式
ax.set_xlabel("DRAG Alpha Coefficient")
ax.set_ylabel("Final Population of |0> State (Fidelity)")
ax.set_title(f"DRAG Suppression of Detuning Error for different repetitions")
ax.grid(True)
ax.legend()
plt.show()


# 绘制 Leakage
fig, ax = plt.subplots(figsize=(12, 8))
colors = plt.get_cmap('viridis')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    # 提取 P0 的数据
    populations_list = results_dict[n]
    p0_values, p1_values, leakage_values = np.array(populations_list).T

    color = colors(i / len(n_values))
    ax.plot(alpha_scan_values, leakage_values, 'o-', ms=4, color=color, label=f'leakage for n={n}')

    # 寻找最优 alpha (使 leakage 最小化的点)
    optimal_alpha_index = np.argmin(leakage_values)
    optimal_alpha = alpha_scan_values[optimal_alpha_index]
    min_leakage = leakage_values[optimal_alpha_index]
    ax.plot(optimal_alpha, min_leakage, '*', color=color, markersize=15, 
            label=f'Optimal for n={n} (alpha={optimal_alpha:.3f})')

# 设置图表格式
ax.set_xlabel("DRAG Alpha Coefficient")
ax.set_ylabel("Final Population of |0> State (Fidelity)")
ax.set_title(f"DRAG Suppression of Detuning Error for different repetitions")
ax.grid(True)
ax.legend()
plt.show()

In [ ]:
# --- 0. 导入库 ---
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, ptrace, Qobj
from qutip import Result
from quantum_sim import DRAGCorrector
# from quantum_sim import ...

# --- 1. 定义物理系统和脉冲参数 ---
#    (为了单元格的独立性，我们在此处重新定义)
D_LEVELS = 3
OMEGA_Q_GHZ = 5.3
ANH_GHZ = -0.212
PULSE_DURATION = 10
PULSE_AMP_RAD_NS = 0.4 * (np.pi / PULSE_DURATION) 
DETUNING_GHZ = -0.08 # -80 MHz 失谐

# --- 2. 仿真控制参数 ---
# 初始状态
PSI0 = basis(D_LEVELS, 0)

# 观测量：|0>, |1> 态布居数和总泄漏
pop0_operator = basis(D_LEVELS, 0) * basis(D_LEVELS, 0).dag()
pop1_operator = basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag()
leakage_operator = Qobj(np.eye(D_LEVELS)) - pop0_operator - pop1_operator

# 固定的实验参数
IDLE_TIME_FIXED = 9.25  # ns, 从之前的仿真中找到的误差最大点

# [核心修改] 我们现在要循环遍历不同的 n 值
n_values = [2, 5, 10] 

# 扫描参数
alpha_scan_values = np.linspace(-2.0, 2.0, 41) # 扫描 alpha 系数

# [核心修改] 准备一个字典来存储每次外层循环的结果
results_dict = {}


# --- 3. 创建共享的模型和仿真器实例 ---
model = DuffingOscillatorModel(d=D_LEVELS, omega_q=OMEGA_Q_GHZ*2*np.pi, anh=ANH_GHZ*2*np.pi)
sim = Simulator(model=model, options={'nsteps': 50000, 'atol': 1e-9, 'rtol': 1e-7})

print("开始多轮 DRAG alpha 系数扫描...")

# --- 4. [核心修改] 增加外层循环来遍历 n_values ---
for n_repetitions in n_values:
    print(f"\\n===== 开始扫描 n = {n_repetitions} 的情况 =====")

    # a) 动态计算本次扫描所需的仿真时间轴
    total_duration = n_repetitions * (PULSE_DURATION + IDLE_TIME_FIXED)
    t_list = np.linspace(0, total_duration, int(total_duration * 50 + 1))

    # b) 定义本次扫描中固定不变的参数
    base_args = {
        'psi0': PSI0,
        't_list': t_list,
        'use_rwa': False,
        'e_ops': [pop0_operator, pop1_operator, leakage_operator]
    }

    # c) 实例化 Scanner
    scanner = Scanner(simulator=sim, base_sim_args=base_args)

    # d) 定义本次扫描的 Setup 函数 (它会捕获当前的 n_repetitions)
    def hd_alpha_setup(alpha: float) -> dict:
        mw_envelope = CosineEnvelope(duration=PULSE_DURATION, amp=PULSE_AMP_RAD_NS)
        idle_envelope = CosineEnvelope(duration=IDLE_TIME_FIXED, amp=0.0)
        sequence_unit = mw_envelope + idle_envelope
        full_envelope = sequence_unit.repeat(n_repetitions)
        
        detuned_mw_sequence = MicrowaveSequence(
            envelope=full_envelope,
            carrier_freq=model.omega_q + (DETUNING_GHZ * 2 * np.pi)
        )
        
        drag_corrector = DRAGCorrector(
            anh=(- DETUNING_GHZ * 2 * np.pi), 
            alpha=alpha
        )
        dragged_sequence = drag_corrector.apply(detuned_mw_sequence)
        
        return {'sequences': {'XY_drive': dragged_sequence}}

    # e) 定义 Post-process 函数
    def final_populations_processor(alpha: float, result: Result) -> tuple:
        p0_final = result.expect[0][-1]
        p1_final = result.expect[1][-1]
        leakage_final = result.expect[2][-1]
        return (p0_final, p1_final, leakage_final)

    # f) 执行对 alpha 的扫描
    _, final_populations_list = scanner.run(
        sweep_values=alpha_scan_values,
        setup_function=hd_alpha_setup,
        post_process_function=final_populations_processor,
        description=f"Scanning alpha for n={n_repetitions}"
    )

    # g) 将结果存储到字典中
    results_dict[n_repetitions] = final_populations_list

print("\\n...所有扫描完成。")

# --- 5. 绘制 P0 ---
fig, ax = plt.subplots(figsize=(12, 8))
colors = plt.get_cmap('viridis')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    # 提取 P0 的数据
    populations_list = results_dict[n]
    p0_values, _, _ = np.array(populations_list).T
    
    color = colors(i / len(n_values))
    ax.plot(alpha_scan_values, p0_values, 'o-', ms=4, color=color, label=f'Pop $|0\\rangle$ for n={n}')
    
    # 寻找最优 alpha (使 P0 最大化的点)
    optimal_alpha_index = np.argmax(p0_values)
    optimal_alpha = alpha_scan_values[optimal_alpha_index]
    max_fidelity = p0_values[optimal_alpha_index]
    ax.plot(optimal_alpha, max_fidelity, '*', color=color, markersize=15, 
            label=f'Optimal for n={n} (alpha={optimal_alpha:.3f})')

# 设置图表格式
ax.set_xlabel("DRAG Alpha Coefficient")
ax.set_ylabel("Final Population of |0> State (Fidelity)")
ax.set_title(f"DRAG Suppression of Detuning Error for different repetitions")
ax.grid(True)
ax.legend()
plt.show()


# 绘制 Leakage
fig, ax = plt.subplots(figsize=(12, 8))
colors = plt.get_cmap('viridis')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    # 提取 P0 的数据
    populations_list = results_dict[n]
    p0_values, p1_values, leakage_values = np.array(populations_list).T

    color = colors(i / len(n_values))
    ax.plot(alpha_scan_values, leakage_values, 'o-', ms=4, color=color, label=f'leakage for n={n}')

    # 寻找最优 alpha (使 leakage 最小化的点)
    optimal_alpha_index = np.argmin(leakage_values)
    optimal_alpha = alpha_scan_values[optimal_alpha_index]
    min_leakage = leakage_values[optimal_alpha_index]
    ax.plot(optimal_alpha, min_leakage, '*', color=color, markersize=15, 
            label=f'Optimal for n={n} (alpha={optimal_alpha:.3f})')

# 设置图表格式
ax.set_xlabel("DRAG Alpha Coefficient")
ax.set_ylabel("Final Population of |0> State (Fidelity)")
ax.set_title(f"DRAG Suppression of Detuning Error for different repetitions")
ax.grid(True)
ax.legend()
plt.show()

### 2.2 使用Half DRAG抑制01跃迁误差

#### 考虑D=2

In [ ]:
# --- 0. 导入必要的库 ---
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, ptrace, Qobj, qeye
from qutip import Result

# 假设所有必要的类都已从您的库中导入
from quantum_sim import (DuffingOscillatorModel, Simulator, Scanner, 
                         CosineEnvelope, MicrowaveSequence, CompositeEnvelope, DRAGCorrector)

# --- 1. 定义物理系统和脉冲参数 ---
# 物理系统参数
D_LEVELS = 2
OMEGA_Q_GHZ = 5.3
ANH_GHZ = -0.212

# 失谐微波脉冲 (MW) 参数
PULSE_DURATION = 10  # ns, 单个微波脉冲的时长
# PULSE_AMP_RAD_NS = (np.pi / PULSE_DURATION) # 幅度大致相当于 1 个 pi/2 脉冲
PULSE_AMP_RAD_NS = (np.pi / PULSE_DURATION) * 0.4
DETUNING_GHZ = -0.08 # -80 MHz 的失谐量

# --- 2. 仿真控制参数 ---
# 初始状态
PSI0 = basis(D_LEVELS, 0)

# 观测量：|1> 态的布居数
pop1_operator = basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag()
# 观测量：|0> 态的布居数
pop0_operator = basis(D_LEVELS, 0) * basis(D_LEVELS, 0).dag()
leakage_operator = qeye(D_LEVELS) - pop0_operator - pop1_operator

# 扫描和循环参数
n_values = [2, 5, 10] # 序列单元的重复次数
# idle_scan_values = np.linspace(0, 1 / OMEGA_Q_GHZ * 5, 21) # 扫描的空闲时间范围 (ns)
idle_scan_values = np.linspace(0, np.abs(1 / DETUNING_GHZ) * 3, 51) # 扫描的空闲时间范围 (ns)


# 准备一个字典来存储每次扫描的结果
results_dict = {}

# --- 3. 创建共享的模型和仿真器实例 ---
model = DuffingOscillatorModel(
    d=D_LEVELS,
    omega_q=OMEGA_Q_GHZ * 2 * np.pi,
    anh=ANH_GHZ * 2 * np.pi
)

sim = Simulator(model=model, options={'nsteps': 50000, 'atol': 1e-9, 'rtol': 1e-7})



print("所有仿真组件已成功创建。准备开始多轮扫描...")

# --- 4. 主循环：遍历不同的脉冲重复次数 n ---

for n in n_values:
    print(f"\\n===== 开始扫描 n = {n} 的情况 =====")

    

    # b) 定义本次扫描中固定不变的参数
    base_args = {
        'psi0': PSI0,
        'use_rwa': False, # 失谐效应在实验室坐标系下更显著和准确
        'e_ops': [pop0_operator, pop1_operator, leakage_operator]
    }

    # c) 实例化 Scanner
    scanner = Scanner(simulator=sim, base_sim_args=base_args)

    # d) 定义本次扫描的 Setup 函数
    def detuning_setup(idle_duration: float) -> dict:
        # 1. 创建基础包络：失谐微波脉冲 (MW) 和 空闲时间 (idle)
        mw_envelope = CosineEnvelope(duration=PULSE_DURATION, amp=PULSE_AMP_RAD_NS)
        idle_envelope = CosineEnvelope(duration=idle_duration, amp=0.0)

        total_duration = n * (PULSE_DURATION + idle_duration)
        t_list = np.linspace(0, total_duration, int(total_duration * 50 + 1))

        # 2. 使用 '+' 和 '.repeat()' 构建完整的序列包络: (MW + idle) * n
        sequence_unit = mw_envelope + idle_envelope
        full_envelope = sequence_unit.repeat(n)
            
        # 3. 基于包络创建带有失谐的微波序列
        detuned_mw_sequence = MicrowaveSequence(
            envelope=full_envelope,
            carrier_freq=model.omega_q + (DETUNING_GHZ * 2 * np.pi)
        )

        # 创建DRAG修正
        drag_corrector = DRAGCorrector(anh=-DETUNING_GHZ * 2 * np.pi, alpha=1)
        detuned_mw_sequence_drag = drag_corrector.apply(detuned_mw_sequence)

        return {'sequences': {'XY_drive': detuned_mw_sequence_drag},
                't_list': t_list
            }

    # e) 定义 Post-process 函数
    def final_pop1_processor(idle_duration: float, result: Result) -> float:
        # 接收了 idle_duration 后，我们直接使用结果的最后一个点即可
        return result.expect[0][-1], result.expect[1][-1], result.expect[2][-1]

    # f) 执行对 idle 时间的扫描
    _, final_populations = scanner.run(
        sweep_values=idle_scan_values,
        setup_function=detuning_setup,
        post_process_function=final_pop1_processor,
        description=f"Scanning idle time for n={n}"
    )

    # g) 将结果存储到字典中
    results_dict[n] = final_populations

print("\\n...所有扫描完成。")

# --- 5. 结果统一可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))
# colors = plt.get_cmap('coolwarm')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    populations = results_dict[n]
    # color = colors(i / len(n_values))
    pop0_data = [info[0] for info in populations]
    pop1_data = [info[1] for info in populations]
    leakage_data = [info[2] for info in populations]

    ax.plot(idle_scan_values, pop0_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |0>')
    ax.plot(idle_scan_values, pop1_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |1>')
    ax.plot(idle_scan_values, leakage_data, 'o-', ms=4, label=f'n = {n} repetitions: Leakage')

# 设置图表格式
ax.set_xlabel("Idle Time (ns)")
ax.set_ylabel("Final Population of |0> State")
ax.set_title(f"Detuned Rabi Oscillation (Detuning = {DETUNING_GHZ*1000} MHz)")
ax.grid(True)
ax.legend()
plt.show()

In [ ]:
(100 - 36/60 *26)

In [ ]:
# --- 5. 结果统一可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))
# colors = plt.get_cmap('coolwarm')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    populations = results_dict[n]
    # color = colors(i / len(n_values))
    pop0_data = [info[0] for info in populations]
    pop1_data = [info[1] for info in populations]
    leakage_data = [info[2] for info in populations]

    ax.plot(idle_scan_values, pop0_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |0>')

# 设置图表格式
ax.set_xlabel("Idle Time (ns)")
ax.set_ylabel("Final Population of |0> State")
ax.set_title(f"Detuned Rabi Oscillation (Detuning = {DETUNING_GHZ*1000} MHz)")
ax.grid(True)
ax.legend()
plt.show()

#### 考虑D=3

##### 采用alpha=1来抑制01跃迁

In [ ]:
# --- 0. 导入必要的库 ---
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, ptrace, Qobj, qeye
from qutip import Result

# 假设所有必要的类都已从您的库中导入
from quantum_sim import (DuffingOscillatorModel, Simulator, Scanner, 
                         CosineEnvelope, MicrowaveSequence, CompositeEnvelope, DRAGCorrector)

# --- 1. 定义物理系统和脉冲参数 ---
# 物理系统参数
D_LEVELS = 3
OMEGA_Q_GHZ = 5.3
ANH_GHZ = -0.212

# 失谐微波脉冲 (MW) 参数
PULSE_DURATION = 10  # ns, 单个微波脉冲的时长
# PULSE_AMP_RAD_NS = (np.pi / PULSE_DURATION) # 幅度大致相当于 1 个 pi/2 脉冲
PULSE_AMP_RAD_NS = (np.pi / PULSE_DURATION) * 0.4
DETUNING_GHZ = -0.08 # -80 MHz 的失谐量

# --- 2. 仿真控制参数 ---
# 初始状态
PSI0 = basis(D_LEVELS, 0)

# 观测量：|1> 态的布居数
pop1_operator = basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag()
# 观测量：|0> 态的布居数
pop0_operator = basis(D_LEVELS, 0) * basis(D_LEVELS, 0).dag()
leakage_operator = qeye(D_LEVELS) - pop0_operator - pop1_operator

# 扫描和循环参数
n_values = [2, 5, 10] # 序列单元的重复次数
# idle_scan_values = np.linspace(0, 1 / OMEGA_Q_GHZ * 5, 21) # 扫描的空闲时间范围 (ns)
idle_scan_values = np.linspace(0, np.abs(1 / DETUNING_GHZ) * 3, 51) # 扫描的空闲时间范围 (ns)


# 准备一个字典来存储每次扫描的结果
results_dict = {}

# --- 3. 创建共享的模型和仿真器实例 ---
model = DuffingOscillatorModel(
    d=D_LEVELS,
    omega_q=OMEGA_Q_GHZ * 2 * np.pi,
    anh=ANH_GHZ * 2 * np.pi
)

sim = Simulator(model=model, options={'nsteps': 50000, 'atol': 1e-9, 'rtol': 1e-7})



print("所有仿真组件已成功创建。准备开始多轮扫描...")

# --- 4. 主循环：遍历不同的脉冲重复次数 n ---

for n in n_values:
    print(f"\\n===== 开始扫描 n = {n} 的情况 =====")

    

    # b) 定义本次扫描中固定不变的参数
    base_args = {
        'psi0': PSI0,
        'use_rwa': False, # 失谐效应在实验室坐标系下更显著和准确
        'e_ops': [pop0_operator, pop1_operator, leakage_operator]
    }

    # c) 实例化 Scanner
    scanner = Scanner(simulator=sim, base_sim_args=base_args)

    # d) 定义本次扫描的 Setup 函数
    def detuning_setup(idle_duration: float) -> dict:
        # 1. 创建基础包络：失谐微波脉冲 (MW) 和 空闲时间 (idle)
        mw_envelope = CosineEnvelope(duration=PULSE_DURATION, amp=PULSE_AMP_RAD_NS)
        idle_envelope = CosineEnvelope(duration=idle_duration, amp=0.0)

        total_duration = n * (PULSE_DURATION + idle_duration)
        t_list = np.linspace(0, total_duration, int(total_duration * 50 + 1))

        # 2. 使用 '+' 和 '.repeat()' 构建完整的序列包络: (MW + idle) * n
        sequence_unit = mw_envelope + idle_envelope
        full_envelope = sequence_unit.repeat(n)
            
        # 3. 基于包络创建带有失谐的微波序列
        detuned_mw_sequence = MicrowaveSequence(
            envelope=full_envelope,
            carrier_freq=model.omega_q + (DETUNING_GHZ * 2 * np.pi)
        )

        # 创建DRAG修正
        drag_corrector = DRAGCorrector(anh=-DETUNING_GHZ * 2 * np.pi, alpha=1)
        detuned_mw_sequence_drag = drag_corrector.apply(detuned_mw_sequence)

        return {'sequences': {'XY_drive': detuned_mw_sequence_drag},
                't_list': t_list
            }

    # e) 定义 Post-process 函数
    def final_pop1_processor(idle_duration: float, result: Result) -> float:
        # 接收了 idle_duration 后，我们直接使用结果的最后一个点即可
        return result.expect[0][-1], result.expect[1][-1], result.expect[2][-1]

    # f) 执行对 idle 时间的扫描
    _, final_populations = scanner.run(
        sweep_values=idle_scan_values,
        setup_function=detuning_setup,
        post_process_function=final_pop1_processor,
        description=f"Scanning idle time for n={n}"
    )

    # g) 将结果存储到字典中
    results_dict[n] = final_populations

print("\\n...所有扫描完成。")

# --- 5. 结果统一可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))
# colors = plt.get_cmap('coolwarm')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    populations = results_dict[n]
    # color = colors(i / len(n_values))
    pop0_data = [info[0] for info in populations]
    pop1_data = [info[1] for info in populations]
    leakage_data = [info[2] for info in populations]

    ax.plot(idle_scan_values, pop0_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |0>')
    # ax.plot(idle_scan_values, pop1_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |1>')
    # ax.plot(idle_scan_values, leakage_data, 'o-', ms=4, label=f'n = {n} repetitions: Leakage')

# 设置图表格式
ax.set_xlabel("Idle Time (ns)")
ax.set_ylabel("Final Population of |0> State")
ax.set_title(f"Detuned Rabi Oscillation (Detuning = {DETUNING_GHZ*1000} MHz)")
ax.grid(True)
ax.legend()
plt.show()

In [ ]:
# --- 5. 结果统一可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))
# colors = plt.get_cmap('coolwarm')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    populations = results_dict[n]
    # color = colors(i / len(n_values))
    pop0_data = [info[0] for info in populations]
    pop1_data = [info[1] for info in populations]
    leakage_data = [info[2] for info in populations]

    # ax.plot(idle_scan_values, pop0_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |0>')
    # ax.plot(idle_scan_values, pop1_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |1>')
    ax.plot(idle_scan_values, leakage_data, 'o-', ms=4, label=f'n = {n} repetitions: Leakage')

# 设置图表格式
ax.set_xlabel("Idle Time (ns)")
ax.set_ylabel("Final Population of |0> State")
ax.set_title(f"Detuned Rabi Oscillation (Detuning = {DETUNING_GHZ*1000} MHz)")
ax.grid(True)
ax.legend()
plt.show()

##### 采用alpha=-1.2来抑制12跃迁

In [ ]:
# --- 0. 导入必要的库 ---
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, ptrace, Qobj, qeye
from qutip import Result

# 假设所有必要的类都已从您的库中导入
from quantum_sim import (DuffingOscillatorModel, Simulator, Scanner, 
                         CosineEnvelope, MicrowaveSequence, CompositeEnvelope, DRAGCorrector)

# --- 1. 定义物理系统和脉冲参数 ---
# 物理系统参数
D_LEVELS = 3
OMEGA_Q_GHZ = 5.3
ANH_GHZ = -0.212

# 失谐微波脉冲 (MW) 参数
PULSE_DURATION = 10  # ns, 单个微波脉冲的时长
# PULSE_AMP_RAD_NS = (np.pi / PULSE_DURATION) # 幅度大致相当于 1 个 pi/2 脉冲
PULSE_AMP_RAD_NS = (np.pi / PULSE_DURATION) * 0.4
DETUNING_GHZ = -0.08 # -80 MHz 的失谐量

# --- 2. 仿真控制参数 ---
# 初始状态
PSI0 = basis(D_LEVELS, 0)

# 观测量：|1> 态的布居数
pop1_operator = basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag()
# 观测量：|0> 态的布居数
pop0_operator = basis(D_LEVELS, 0) * basis(D_LEVELS, 0).dag()
leakage_operator = qeye(D_LEVELS) - pop0_operator - pop1_operator

# 扫描和循环参数
n_values = [2, 5, 10] # 序列单元的重复次数
# idle_scan_values = np.linspace(0, 1 / OMEGA_Q_GHZ * 5, 21) # 扫描的空闲时间范围 (ns)
idle_scan_values = np.linspace(0, np.abs(1 / DETUNING_GHZ) * 3, 51) # 扫描的空闲时间范围 (ns)


# 准备一个字典来存储每次扫描的结果
results_dict = {}

# --- 3. 创建共享的模型和仿真器实例 ---
model = DuffingOscillatorModel(
    d=D_LEVELS,
    omega_q=OMEGA_Q_GHZ * 2 * np.pi,
    anh=ANH_GHZ * 2 * np.pi
)

sim = Simulator(model=model, options={'nsteps': 50000, 'atol': 1e-9, 'rtol': 1e-7})



print("所有仿真组件已成功创建。准备开始多轮扫描...")

# --- 4. 主循环：遍历不同的脉冲重复次数 n ---

for n in n_values:
    print(f"\\n===== 开始扫描 n = {n} 的情况 =====")

    

    # b) 定义本次扫描中固定不变的参数
    base_args = {
        'psi0': PSI0,
        'use_rwa': False, # 失谐效应在实验室坐标系下更显著和准确
        'e_ops': [pop0_operator, pop1_operator, leakage_operator]
    }

    # c) 实例化 Scanner
    scanner = Scanner(simulator=sim, base_sim_args=base_args)

    # d) 定义本次扫描的 Setup 函数
    def detuning_setup(idle_duration: float) -> dict:
        # 1. 创建基础包络：失谐微波脉冲 (MW) 和 空闲时间 (idle)
        mw_envelope = CosineEnvelope(duration=PULSE_DURATION, amp=PULSE_AMP_RAD_NS)
        idle_envelope = CosineEnvelope(duration=idle_duration, amp=0.0)

        total_duration = n * (PULSE_DURATION + idle_duration)
        t_list = np.linspace(0, total_duration, int(total_duration * 50 + 1))

        # 2. 使用 '+' 和 '.repeat()' 构建完整的序列包络: (MW + idle) * n
        sequence_unit = mw_envelope + idle_envelope
        full_envelope = sequence_unit.repeat(n)
            
        # 3. 基于包络创建带有失谐的微波序列
        detuned_mw_sequence = MicrowaveSequence(
            envelope=full_envelope,
            carrier_freq=model.omega_q + (DETUNING_GHZ * 2 * np.pi)
        )

        # 创建DRAG修正
        drag_corrector = DRAGCorrector(anh=-DETUNING_GHZ * 2 * np.pi, alpha=-1.2)
        detuned_mw_sequence_drag = drag_corrector.apply(detuned_mw_sequence)

        return {'sequences': {'XY_drive': detuned_mw_sequence_drag},
                't_list': t_list
            }

    # e) 定义 Post-process 函数
    def final_pop1_processor(idle_duration: float, result: Result) -> float:
        # 接收了 idle_duration 后，我们直接使用结果的最后一个点即可
        return result.expect[0][-1], result.expect[1][-1], result.expect[2][-1]

    # f) 执行对 idle 时间的扫描
    _, final_populations = scanner.run(
        sweep_values=idle_scan_values,
        setup_function=detuning_setup,
        post_process_function=final_pop1_processor,
        description=f"Scanning idle time for n={n}"
    )

    # g) 将结果存储到字典中
    results_dict[n] = final_populations

print("\\n...所有扫描完成。")

# --- 5. 结果统一可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))
# colors = plt.get_cmap('coolwarm')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    populations = results_dict[n]
    # color = colors(i / len(n_values))
    pop0_data = [info[0] for info in populations]
    pop1_data = [info[1] for info in populations]
    leakage_data = [info[2] for info in populations]

    ax.plot(idle_scan_values, pop0_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |0>')
    # ax.plot(idle_scan_values, pop1_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |1>')
    # ax.plot(idle_scan_values, leakage_data, 'o-', ms=4, label=f'n = {n} repetitions: Leakage')

# 设置图表格式
ax.set_xlabel("Idle Time (ns)")
ax.set_ylabel("Final Population of |0> State")
ax.set_title(f"Detuned Rabi Oscillation (Detuning = {DETUNING_GHZ*1000} MHz)")
ax.grid(True)
ax.legend()
plt.show()

In [ ]:
# --- 5. 结果统一可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))
# colors = plt.get_cmap('coolwarm')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    populations = results_dict[n]
    # color = colors(i / len(n_values))
    pop0_data = [info[0] for info in populations]
    pop1_data = [info[1] for info in populations]
    leakage_data = [info[2] for info in populations]

    # ax.plot(idle_scan_values, pop0_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |0>')
    ax.plot(idle_scan_values, pop1_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |1>')
    # ax.plot(idle_scan_values, leakage_data, 'o-', ms=4, label=f'n = {n} repetitions: Leakage')

# 设置图表格式
ax.set_xlabel("Idle Time (ns)")
ax.set_ylabel("Final Population of |0> State")
ax.set_title(f"Detuned Rabi Oscillation (Detuning = {DETUNING_GHZ*1000} MHz)")
ax.grid(True)
ax.legend()
plt.show()

In [ ]:
# --- 5. 结果统一可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))
# colors = plt.get_cmap('coolwarm')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    populations = results_dict[n]
    # color = colors(i / len(n_values))
    pop0_data = [info[0] for info in populations]
    pop1_data = [info[1] for info in populations]
    leakage_data = [info[2] for info in populations]

    # ax.plot(idle_scan_values, pop0_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |0>')
    # ax.plot(idle_scan_values, pop1_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |1>')
    ax.plot(idle_scan_values, leakage_data, 'o-', ms=4, label=f'n = {n} repetitions: Leakage')

# 设置图表格式
ax.set_xlabel("Idle Time (ns)")
ax.set_ylabel("Final Population of |0> State")
ax.set_title(f"Detuned Rabi Oscillation (Detuning = {DETUNING_GHZ*1000} MHz)")
ax.grid(True)
ax.legend()
plt.show()

##### 采用alpha=-0.5来抑制12跃迁

In [ ]:
# --- 0. 导入必要的库 ---
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, ptrace, Qobj, qeye
from qutip import Result

# 假设所有必要的类都已从您的库中导入
from quantum_sim import (DuffingOscillatorModel, Simulator, Scanner, 
                         CosineEnvelope, MicrowaveSequence, CompositeEnvelope, DRAGCorrector)

# --- 1. 定义物理系统和脉冲参数 ---
# 物理系统参数
D_LEVELS = 3
OMEGA_Q_GHZ = 5.3
ANH_GHZ = -0.212

# 失谐微波脉冲 (MW) 参数
PULSE_DURATION = 10  # ns, 单个微波脉冲的时长
# PULSE_AMP_RAD_NS = (np.pi / PULSE_DURATION) # 幅度大致相当于 1 个 pi/2 脉冲
PULSE_AMP_RAD_NS = (np.pi / PULSE_DURATION) * 0.4
DETUNING_GHZ = -0.08 # -80 MHz 的失谐量

# --- 2. 仿真控制参数 ---
# 初始状态
PSI0 = basis(D_LEVELS, 0)

# 观测量：|1> 态的布居数
pop1_operator = basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag()
# 观测量：|0> 态的布居数
pop0_operator = basis(D_LEVELS, 0) * basis(D_LEVELS, 0).dag()
leakage_operator = qeye(D_LEVELS) - pop0_operator - pop1_operator

# 扫描和循环参数
n_values = [2, 5, 10] # 序列单元的重复次数
# idle_scan_values = np.linspace(0, 1 / OMEGA_Q_GHZ * 5, 21) # 扫描的空闲时间范围 (ns)
idle_scan_values = np.linspace(0, np.abs(1 / DETUNING_GHZ) * 3, 51) # 扫描的空闲时间范围 (ns)


# 准备一个字典来存储每次扫描的结果
results_dict = {}

# --- 3. 创建共享的模型和仿真器实例 ---
model = DuffingOscillatorModel(
    d=D_LEVELS,
    omega_q=OMEGA_Q_GHZ * 2 * np.pi,
    anh=ANH_GHZ * 2 * np.pi
)

sim = Simulator(model=model, options={'nsteps': 50000, 'atol': 1e-9, 'rtol': 1e-7})



print("所有仿真组件已成功创建。准备开始多轮扫描...")

# --- 4. 主循环：遍历不同的脉冲重复次数 n ---

for n in n_values:
    print(f"\\n===== 开始扫描 n = {n} 的情况 =====")

    

    # b) 定义本次扫描中固定不变的参数
    base_args = {
        'psi0': PSI0,
        'use_rwa': False, # 失谐效应在实验室坐标系下更显著和准确
        'e_ops': [pop0_operator, pop1_operator, leakage_operator]
    }

    # c) 实例化 Scanner
    scanner = Scanner(simulator=sim, base_sim_args=base_args)

    # d) 定义本次扫描的 Setup 函数
    def detuning_setup(idle_duration: float) -> dict:
        # 1. 创建基础包络：失谐微波脉冲 (MW) 和 空闲时间 (idle)
        mw_envelope = CosineEnvelope(duration=PULSE_DURATION, amp=PULSE_AMP_RAD_NS)
        idle_envelope = CosineEnvelope(duration=idle_duration, amp=0.0)

        total_duration = n * (PULSE_DURATION + idle_duration)
        t_list = np.linspace(0, total_duration, int(total_duration * 50 + 1))

        # 2. 使用 '+' 和 '.repeat()' 构建完整的序列包络: (MW + idle) * n
        sequence_unit = mw_envelope + idle_envelope
        full_envelope = sequence_unit.repeat(n)
            
        # 3. 基于包络创建带有失谐的微波序列
        detuned_mw_sequence = MicrowaveSequence(
            envelope=full_envelope,
            carrier_freq=model.omega_q + (DETUNING_GHZ * 2 * np.pi)
        )

        # 创建DRAG修正
        drag_corrector = DRAGCorrector(anh=-DETUNING_GHZ * 2 * np.pi, alpha=-0.5)
        detuned_mw_sequence_drag = drag_corrector.apply(detuned_mw_sequence)

        return {'sequences': {'XY_drive': detuned_mw_sequence_drag},
                't_list': t_list
            }

    # e) 定义 Post-process 函数
    def final_pop1_processor(idle_duration: float, result: Result) -> float:
        # 接收了 idle_duration 后，我们直接使用结果的最后一个点即可
        return result.expect[0][-1], result.expect[1][-1], result.expect[2][-1]

    # f) 执行对 idle 时间的扫描
    _, final_populations = scanner.run(
        sweep_values=idle_scan_values,
        setup_function=detuning_setup,
        post_process_function=final_pop1_processor,
        description=f"Scanning idle time for n={n}"
    )

    # g) 将结果存储到字典中
    results_dict[n] = final_populations

print("\\n...所有扫描完成。")

# --- 5. 结果统一可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))
# colors = plt.get_cmap('coolwarm')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    populations = results_dict[n]
    # color = colors(i / len(n_values))
    pop0_data = [info[0] for info in populations]
    pop1_data = [info[1] for info in populations]
    leakage_data = [info[2] for info in populations]

    ax.plot(idle_scan_values, pop0_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |0>')
    # ax.plot(idle_scan_values, pop1_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |1>')
    # ax.plot(idle_scan_values, leakage_data, 'o-', ms=4, label=f'n = {n} repetitions: Leakage')

# 设置图表格式
ax.set_xlabel("Idle Time (ns)")
ax.set_ylabel("Final Population of |0> State")
ax.set_title(f"Detuned Rabi Oscillation (Detuning = {DETUNING_GHZ*1000} MHz)")
ax.grid(True)
ax.legend()
plt.show()

In [ ]:
# --- 5. 结果统一可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))
# colors = plt.get_cmap('coolwarm')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    populations = results_dict[n]
    # color = colors(i / len(n_values))
    pop0_data = [info[0] for info in populations]
    pop1_data = [info[1] for info in populations]
    leakage_data = [info[2] for info in populations]

    # ax.plot(idle_scan_values, pop0_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |0>')
    ax.plot(idle_scan_values, pop1_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |1>')
    # ax.plot(idle_scan_values, leakage_data, 'o-', ms=4, label=f'n = {n} repetitions: Leakage')

# 设置图表格式
ax.set_xlabel("Idle Time (ns)")
ax.set_ylabel("Final Population of |0> State")
ax.set_title(f"Detuned Rabi Oscillation (Detuning = {DETUNING_GHZ*1000} MHz)")
ax.grid(True)
ax.legend()
plt.show()

In [ ]:
# --- 5. 结果统一可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))
# colors = plt.get_cmap('coolwarm')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    populations = results_dict[n]
    # color = colors(i / len(n_values))
    pop0_data = [info[0] for info in populations]
    pop1_data = [info[1] for info in populations]
    leakage_data = [info[2] for info in populations]

    # ax.plot(idle_scan_values, pop0_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |0>')
    # ax.plot(idle_scan_values, pop1_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |1>')
    ax.plot(idle_scan_values, leakage_data, 'o-', ms=4, label=f'n = {n} repetitions: Leakage')

# 设置图表格式
ax.set_xlabel("Idle Time (ns)")
ax.set_ylabel("Final Population of |0> State")
ax.set_title(f"Detuned Rabi Oscillation (Detuning = {DETUNING_GHZ*1000} MHz)")
ax.grid(True)
ax.legend()
plt.show()

##### 采用alpha=-1来抑制12跃迁

In [ ]:
# --- 0. 导入必要的库 ---
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, ptrace, Qobj, qeye
from qutip import Result

# 假设所有必要的类都已从您的库中导入
from quantum_sim import (DuffingOscillatorModel, Simulator, Scanner, 
                         CosineEnvelope, MicrowaveSequence, CompositeEnvelope, DRAGCorrector)

# --- 1. 定义物理系统和脉冲参数 ---
# 物理系统参数
D_LEVELS = 3
OMEGA_Q_GHZ = 5.3
ANH_GHZ = -0.212

# 失谐微波脉冲 (MW) 参数
PULSE_DURATION = 10  # ns, 单个微波脉冲的时长
# PULSE_AMP_RAD_NS = (np.pi / PULSE_DURATION) # 幅度大致相当于 1 个 pi/2 脉冲
PULSE_AMP_RAD_NS = (np.pi / PULSE_DURATION) * 0.4
DETUNING_GHZ = -0.08 # -80 MHz 的失谐量

# --- 2. 仿真控制参数 ---
# 初始状态
PSI0 = basis(D_LEVELS, 0)

# 观测量：|1> 态的布居数
pop1_operator = basis(D_LEVELS, 1) * basis(D_LEVELS, 1).dag()
# 观测量：|0> 态的布居数
pop0_operator = basis(D_LEVELS, 0) * basis(D_LEVELS, 0).dag()
leakage_operator = qeye(D_LEVELS) - pop0_operator - pop1_operator

# 扫描和循环参数
n_values = [2, 5, 10] # 序列单元的重复次数
# idle_scan_values = np.linspace(0, 1 / OMEGA_Q_GHZ * 5, 21) # 扫描的空闲时间范围 (ns)
idle_scan_values = np.linspace(0, np.abs(1 / DETUNING_GHZ) * 3, 51) # 扫描的空闲时间范围 (ns)


# 准备一个字典来存储每次扫描的结果
results_dict = {}

# --- 3. 创建共享的模型和仿真器实例 ---
model = DuffingOscillatorModel(
    d=D_LEVELS,
    omega_q=OMEGA_Q_GHZ * 2 * np.pi,
    anh=ANH_GHZ * 2 * np.pi
)

sim = Simulator(model=model, options={'nsteps': 50000, 'atol': 1e-9, 'rtol': 1e-7})



print("所有仿真组件已成功创建。准备开始多轮扫描...")

# --- 4. 主循环：遍历不同的脉冲重复次数 n ---

for n in n_values:
    print(f"\\n===== 开始扫描 n = {n} 的情况 =====")

    

    # b) 定义本次扫描中固定不变的参数
    base_args = {
        'psi0': PSI0,
        'use_rwa': False, # 失谐效应在实验室坐标系下更显著和准确
        'e_ops': [pop0_operator, pop1_operator, leakage_operator]
    }

    # c) 实例化 Scanner
    scanner = Scanner(simulator=sim, base_sim_args=base_args)

    # d) 定义本次扫描的 Setup 函数
    def detuning_setup(idle_duration: float) -> dict:
        # 1. 创建基础包络：失谐微波脉冲 (MW) 和 空闲时间 (idle)
        mw_envelope = CosineEnvelope(duration=PULSE_DURATION, amp=PULSE_AMP_RAD_NS)
        idle_envelope = CosineEnvelope(duration=idle_duration, amp=0.0)

        total_duration = n * (PULSE_DURATION + idle_duration)
        t_list = np.linspace(0, total_duration, int(total_duration * 50 + 1))

        # 2. 使用 '+' 和 '.repeat()' 构建完整的序列包络: (MW + idle) * n
        sequence_unit = mw_envelope + idle_envelope
        full_envelope = sequence_unit.repeat(n)
            
        # 3. 基于包络创建带有失谐的微波序列
        detuned_mw_sequence = MicrowaveSequence(
            envelope=full_envelope,
            carrier_freq=model.omega_q + (DETUNING_GHZ * 2 * np.pi)
        )

        # 创建DRAG修正
        drag_corrector = DRAGCorrector(anh=-DETUNING_GHZ * 2 * np.pi, alpha=-1)
        detuned_mw_sequence_drag = drag_corrector.apply(detuned_mw_sequence)

        return {'sequences': {'XY_drive': detuned_mw_sequence_drag},
                't_list': t_list
            }

    # e) 定义 Post-process 函数
    def final_pop1_processor(idle_duration: float, result: Result) -> float:
        # 接收了 idle_duration 后，我们直接使用结果的最后一个点即可
        return result.expect[0][-1], result.expect[1][-1], result.expect[2][-1]

    # f) 执行对 idle 时间的扫描
    _, final_populations = scanner.run(
        sweep_values=idle_scan_values,
        setup_function=detuning_setup,
        post_process_function=final_pop1_processor,
        description=f"Scanning idle time for n={n}"
    )

    # g) 将结果存储到字典中
    results_dict[n] = final_populations

print("\\n...所有扫描完成。")

# --- 5. 结果统一可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))
# colors = plt.get_cmap('coolwarm')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    populations = results_dict[n]
    # color = colors(i / len(n_values))
    pop0_data = [info[0] for info in populations]
    pop1_data = [info[1] for info in populations]
    leakage_data = [info[2] for info in populations]

    ax.plot(idle_scan_values, pop0_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |0>')
    # ax.plot(idle_scan_values, pop1_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |1>')
    # ax.plot(idle_scan_values, leakage_data, 'o-', ms=4, label=f'n = {n} repetitions: Leakage')

# 设置图表格式
ax.set_xlabel("Idle Time (ns)")
ax.set_ylabel("Final Population of |0> State")
ax.set_title(f"Detuned Rabi Oscillation (Detuning = {DETUNING_GHZ*1000} MHz)")
ax.grid(True)
ax.legend()
plt.show()

In [ ]:
# --- 5. 结果统一可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))
# colors = plt.get_cmap('coolwarm')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    populations = results_dict[n]
    # color = colors(i / len(n_values))
    pop0_data = [info[0] for info in populations]
    pop1_data = [info[1] for info in populations]
    leakage_data = [info[2] for info in populations]

    # ax.plot(idle_scan_values, pop0_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |0>')
    ax.plot(idle_scan_values, pop1_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |1>')
    # ax.plot(idle_scan_values, leakage_data, 'o-', ms=4, label=f'n = {n} repetitions: Leakage')

# 设置图表格式
ax.set_xlabel("Idle Time (ns)")
ax.set_ylabel("Final Population of |0> State")
ax.set_title(f"Detuned Rabi Oscillation (Detuning = {DETUNING_GHZ*1000} MHz)")
ax.grid(True)
ax.legend()
plt.show()

In [ ]:
# --- 5. 结果统一可视化 ---
fig, ax = plt.subplots(figsize=(12, 8))
# colors = plt.get_cmap('coolwarm')

# 遍历存储的结果并绘图
for i, n in enumerate(n_values):
    populations = results_dict[n]
    # color = colors(i / len(n_values))
    pop0_data = [info[0] for info in populations]
    pop1_data = [info[1] for info in populations]
    leakage_data = [info[2] for info in populations]

    # ax.plot(idle_scan_values, pop0_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |0>')
    # ax.plot(idle_scan_values, pop1_data, 'o-', ms=4, label=f'n = {n} repetitions: Pop |1>')
    ax.plot(idle_scan_values, leakage_data, 'o-', ms=4, label=f'n = {n} repetitions: Leakage')

# 设置图表格式
ax.set_xlabel("Idle Time (ns)")
ax.set_ylabel("Final Population of |0> State")
ax.set_title(f"Detuned Rabi Oscillation (Detuning = {DETUNING_GHZ*1000} MHz)")
ax.grid(True)
ax.legend()
plt.show()